# 課題3：タイタニック号乗客の生存状況の分類モデル作成

本課題では、`titanic` というデータセットを使います。これは、1912年に発生したタイタニック号の沈没事故における乗客の生存状況に関するデータセットです。元々は、[Encyclopedia Titanica](https://www.encyclopedia-titanica.org/)で掲載されたデータと言われており、このデータセットを組み込んだPythonのライブラリも複数あります。

今回は、`seaborn` のライブラリに組み込まれた `titanic` のデータセットを使います。各セルに入っているコメントの下に、実行するコードを記入してください。わからない場合は、ここまでのレッスン内容や各種ライブラリの公式ドキュメントを参照しましょう。

## 1. 必要なライブラリのimport

In [87]:
# 必要なライブラリのimport（変更しないでください）
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

from sklearn.model_selection import train_test_split

# DataFrameですべての列を表示する設定（変更しないでください）
pd.options.display.max_columns = None

## 2. データの読み込み

seabornに添付のデータセットから「titanic」を読み込み、内容を確認します。

In [54]:
# seabornからtitanicのデータセットを読み込む（変更しないでください）
dataset = sns.load_dataset("titanic")

`sns.load_dataset()` で読み込んだデータは、pandasのDataFrameになっています。

In [55]:
# datasetの先頭5件を確認
dataset.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True


### 使用する列の指定

今回は `survived, pclass, sex, age, sibsp, parch, fare, embarked` の列を使用します。

#### 参考:各列の説明

- `survived`: 生存区分（0:死亡, 1:生存）
- `pclass`: チケットクラス
- `sex`: 性別（male:男性, female:女性）
- `age`: 年齢
- `sibsp`: 同乗している兄弟や配偶者の数
- `parch`: 同乗している親や子供の数
- `fare`: 料金
- `embarked`: 乗船した港（頭文字）
- `class`: 客室クラス
- `who`: 性別（man:男性, woman:女性）
- `adult_male`: 成人男性ならTrue
- `deck`: 事故の際にどのデッキにいたか
- `embark_town`: 乗船した港名
- `alive`: 生存区分（no:死亡, yes:生存）
- `alone`: 1人で乗船したか

In [56]:
# datasetから「survived, pclass, sex, age, sibsp, parch, fare, embarked」の列を取得して
# datasetに代入（上書き）する
"""
survived`: 生存区分（0:死亡, 1:生存）
pclass`: チケットクラス
sex`: 性別（male:男性, female:女性）
age`: 年齢
sibsp`: 同乗している兄弟や配偶者の数
parch`: 同乗している親や子供の数
fare`: 料金
embarked`: 乗船した港（頭文字）
"""
dataset = dataset[["survived", "pclass", "sex", "age", "sibsp", "parch", "fare", "embarked"]]

In [57]:
# 改めてdatasetの先頭5件を表示
dataset.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked
0,0,3,male,22.0,1,0,7.2500,S
1,1,1,female,38.0,1,0,71.2833,C
2,1,3,female,26.0,0,0,7.9250,S
3,1,1,female,35.0,1,0,53.1000,S
4,0,3,male,35.0,0,0,8.0500,S


## 3. データの前処理

### 要約統計量の表示

In [58]:
# 要約統計量を表示
# DataFrameの情報（データ型・非欠損数）を表示
dataset.info()

# 数値・カテゴリの要約統計量を表示（カテゴリ列も含める）
dataset.describe(include='all')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   survived  891 non-null    int64  
 1   pclass    891 non-null    int64  
 2   sex       891 non-null    object 
 3   age       714 non-null    float64
 4   sibsp     891 non-null    int64  
 5   parch     891 non-null    int64  
 6   fare      891 non-null    float64
 7   embarked  889 non-null    object 
dtypes: float64(2), int64(4), object(2)
memory usage: 55.8+ KB


,survived,pclass,sex,age,sibsp,parch,fare,embarked
count,891.000000,891.000000,891,714.000000,891.000000,891.000000,891.000000,889
unique,NaN,NaN,2,NaN,NaN,NaN,NaN,3
top,NaN,NaN,male,NaN,NaN,NaN,NaN,S
freq,NaN,NaN,577,NaN,NaN,NaN,NaN,644
mean,0.383838,2.308642,NaN,29.699118,0.523008,0.381594,32.204208,NaN
std,0.486592,0.836071,NaN,14.526497,1.102743,0.806057,49.693429,NaN
min,0.000000,1.000000,NaN,0.420000,0.000000,0.000000,0.000000,NaN
25%,0.000000,2.000000,NaN,20.125000,0.000000,0.000000,7.910400,NaN
50%,0.000000,3.000000,NaN,28.000000,0.000000,0.000000,14.454200,NaN
75%,1.000000,3.000000,NaN,38.000000,1.000000,0.000000,31.000000,NaN


### 欠損値の確認と補完

In [59]:
# 各列の欠損値の数を確認
missing_counts = dataset.isnull().sum()
missing_pct = (dataset.isnull().mean() * 100).round(2)
missing_df = pd.concat([missing_counts, missing_pct], axis=1)
missing_df.columns = ["missing_count", "missing_pct"]
missing_df

,missing_count,missing_pct
survived,0,0.00
pclass,0,0.00
sex,0,0.00
age,177,19.87
sibsp,0,0.00
parch,0,0.00
fare,0,0.00
embarked,2,0.22


ageの欠損値は平均値で補完します。

In [60]:
# 補完前に欠損数確認
print("補完前のage欠損数=", dataset['age'].isnull().sum())

補完前のage欠損数= 177


In [61]:
# ageの欠損値を、ageの平均値で補完する
# 小数点第一位まで丸めたものをセット
mean_age = round(dataset['age'].mean(),1)
dataset['age'] = dataset['age'].fillna(mean_age)

# 補完後の欠損数（確認用）
print("mean_age=", mean_age)
print("補完後のage欠損数=", dataset['age'].isnull().sum())

mean_age= 29.7
補完後のage欠損数= 0


embarkedの欠損値は、もっとも乗船者数の多い港で補完します。

その方法はいくつかありますが、ここではその1つとして、DataFrameの特定の1列（Series）が持つ `value_counts()` メソッドを紹介します。このメソッドを実行すると、その列が持つ値ごとのデータ数がわかります。

参考：[pandas.Series.value_counts](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.value_counts.html)

In [62]:
# 補完前に欠損数確認
print("補完前のembarked欠損数=", dataset['embarked'].isnull().sum())

補完前のembarked欠損数= 2


In [63]:
# 乗船者数の多い港を value_counts メソッドで確認
print("ーーーーーーーー乗船者数の港（embarked）の分布ーーーーーーーー")
print(dataset['embarked'].value_counts(dropna=False))

ーーーーーーーー乗船者数の港（embarked）の分布ーーーーーーーー
embarked
S      644
C      168
Q       77
NaN      2
Name: count, dtype: int64


`values_count()` の結果を見て、もっとも乗船者数の多い港の文字で欠損値を埋めるようにします。

In [64]:
# embarkedの欠損値をもっとも乗船者数の多い港にて補完
# embarkedの欠損値をもっとも乗船者数の多い港（最頻値）で補完
most_frequent_port = dataset['embarked'].mode()[0]
dataset['embarked'] = dataset['embarked'].fillna(most_frequent_port)

上記の処理により、欠損値がなくなったかを確認しましょう。

In [65]:
# 欠損値の数を確認し、補完後の欠損値が0であることを確認
print("補完後のembarked欠損数=", dataset['embarked'].isnull().sum())
print(dataset['embarked'].value_counts(dropna=False))

補完後のembarked欠損数= 0
embarked
S    646
C    168
Q     77
Name: count, dtype: int64


### ダミー変数への変換

sexとembarkedをダミー変数に変換します。

In [66]:
# datasetのsexとembarkedをダミー変数に変換してdataset2に代入する
dataset2 = pd.get_dummies(dataset, columns=['sex', 'embarked'])

In [67]:
# dataset2のデータの最初の5行を表示
dataset2.head()

,survived,pclass,age,sibsp,parch,fare,sex_female,sex_male,embarked_C,embarked_Q,embarked_S
0,0,3,22.0,1,0,7.2500,False,True,False,False,True
1,1,1,38.0,1,0,71.2833,True,False,True,False,False
2,1,3,26.0,0,0,7.9250,True,False,False,False,True
3,1,1,35.0,1,0,53.1000,True,False,False,False,True
4,0,3,35.0,0,0,8.0500,False,True,False,False,True


In [68]:
# dataset2のnull値を確認
missing_counts2 = dataset2.isnull().sum()
missing_pct2 = (dataset2.isnull().mean() * 100).round(2)
missing_df2 = pd.concat([missing_counts2, missing_pct2], axis=1)
missing_df2.columns = ["missing_count2", "missing_pct2"]
missing_df2

,missing_count2,missing_pct2
survived,0,0.0
pclass,0,0.0
age,0,0.0
sibsp,0,0.0
parch,0,0.0
fare,0,0.0
sex_female,0,0.0
sex_male,0,0.0
embarked_C,0,0.0
embarked_Q,0,0.0


## 4. 目的変数と説明変数の選択

ここでは、以下の列を使用します。

- 目的変数: `survived`
- 説明変数: それ以外

dataset2より目的変数と説明変数に該当する列を取得してnumpy配列に変換し、変数YとXに格納します。列の除外には、DataFrameの `drop` を使います。`データフレーム.drop(columns=除外したい列名)` です。

In [69]:
# Y:目的変数に該当する列
Y = dataset2['survived'].to_numpy()

# X:説明変数に該当する列。dataset2からsurvivedを除外
X = dataset2.drop(columns=['survived']).to_numpy()

In [70]:
# YとXの形状を確認
print("Yの形状:", Y.shape)
print("Xの形状:", X.shape)

Yの形状: (891,)
Xの形状: (891, 10)


## 5. データの分割

この課題ではホールドアウト法でデータを分割します。

In [71]:
# X と Y を 機械学習用データとテストデータに7:3に分ける(X_train, X_test, Y_train, Y_test)
X_train, X_test, Y_train, Y_test = train_test_split(
    X, Y, test_size=0.3, random_state=0, stratify=Y
)

In [72]:
# 機械学習用データを、学習データと検証データに7:3に分ける(X_train, X_valid, Y_train, Y_valid)
X_train, X_valid, Y_train, Y_valid = train_test_split(
    X_train, Y_train, test_size=0.3, random_state=0, stratify=Y_train
)

In [73]:
# 形状を確認:X_train, X_valid, X_test, Y_train, Y_valid, Y_test
print("X_train:", X_train.shape)
print("X_valid:", X_valid.shape)
print("X_test :", X_test.shape)
print("Y_train:", Y_train.shape)
print("Y_valid:", Y_valid.shape)
print("Y_test :", Y_test.shape)

X_train: (436, 10)
X_valid: (187, 10)
X_test : (268, 10)
Y_train: (436,)
Y_valid: (187,)
Y_test : (268,)


## 6. モデルの選択

ロジスティック回帰と決定木、ランダムフォレスト、SVMの4つのモデルを作成して比較します。

In [74]:
# 必要なライブラリの追加import（変更しないでください）
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import f1_score

モデルの評価（性能の比較）には、F1値を使ってください。以下には1つだけセルを用意していますが、モデルを4つ作って比較する処理のためにセルを増やしてもかまいません。

In [75]:
# 4つのモデルを作成し、それぞれのF1値を出力する
"""
■averageの説明
macro：各クラスごとに計算した F1 を単純平均（重みなし）する指定
micro：全サンプルで TP/FP/FN を合算してから F1 を計算（サポート数の大きいクラスが効く）
weighted：各クラスの F1 を サポート数で重み付き平均（不均衡にやや頑健、ただし少数クラスの影響は薄まる）
binary：2値分類の陽性クラスのみで F1 を計算（pos_label が対象）。なお、指定しない場合（デフォルト）は average='binary' 
"""
models = {
    "LogisticRegression": LogisticRegression(max_iter=1000, random_state=0),  # ロジスティック回帰
    "DecisionTree"      : DecisionTreeClassifier(random_state=0), # 決定木
    "RandomForest"      : RandomForestClassifier(random_state=0), # ランダムフォレスト
    "SVM"               : SVC(random_state=0) # SVM
}

for name, model in models.items():
    model.fit(X_train, Y_train)
    y_pred = model.predict(X_valid)
    print(f"{name} F1: {f1_score(Y_valid, y_pred, average='macro'):.4f}")

LogisticRegression F1: 0.7869
DecisionTree F1: 0.7783
RandomForest F1: 0.7574
SVM F1: 0.6781


## 7. パラメータのチューニング

GridSearchCVを使い、性能の良かったランダムフォレストのパラメータのチューニングを行ないます。パラメータの候補については、レッスン本編を参考にしてください。

In [76]:
# 必要なライブラリの追加import（変更しないでください）
from sklearn.model_selection import GridSearchCV

In [77]:
# 性能の良かったモデルを作成
## お題がランダムフォレストになっているので、ランダムフォレストとする
rf = RandomForestClassifier(random_state=0)
print("rf=",rf)

rf= RandomForestClassifier(random_state=0)


In [78]:
# パラメータの指定
param_grid = {
    "n_estimators": [100, 200, 300],
    "max_depth": [None, 5, 10, 15],
    "min_samples_split": [2, 5, 10],
    "min_samples_leaf": [1, 2, 4],
    "max_features": ["sqrt", "log2", None],
    "class_weight": [None, "balanced"]
}

In [79]:
# グリッドサーチのオブジェクトを作成
gs = GridSearchCV(
    estimator=rf,
    param_grid=param_grid,
    scoring="f1_macro",
    cv=5,
    n_jobs=-1,
    verbose=0
)

In [80]:
# データの分割:機械学習用データを学習と検証に分けるのはクロスバリデーションで行ってくれる
# （Xg_train, Xg_test, Yg_train, Yg_test）
Xg_train, Xg_test, Yg_train, Yg_test = train_test_split(
    X_train, Y_train, test_size=0.2, random_state=0, stratify=Y_train
)

In [81]:
# グリッドサーチを実行する
gs.fit(Xg_train, Yg_train)

GridSearchCV(cv=5, estimator=RandomForestClassifier(random_state=0), n_jobs=-1,
             param_grid={'class_weight': [None, 'balanced'],
                         'max_depth': [None, 5, 10, 15],
                         'max_features': ['sqrt', 'log2', None],
                         'min_samples_leaf': [1, 2, 4],
                         'min_samples_split': [2, 5, 10],
                         'n_estimators': [100, 200, 300]},
             scoring='f1_macro')

In [82]:
# 最適なパラメータを表示
print("Best CV F1(macro):", f"{gs.best_score_:.4f}")
print("Best Params:", gs.best_params_)

Best CV F1(macro): 0.7996
Best Params: {'class_weight': None, 'max_depth': 10, 'max_features': 'sqrt', 'min_samples_leaf': 1, 'min_samples_split': 5, 'n_estimators': 200}


ここで得たパラメータをもとに、モデルを再度作成します。

In [83]:
# 最適なパラメータによるモデルの作成
best_rf = RandomForestClassifier(random_state=0, **gs.best_params_)

# モデルの学習
best_rf.fit(Xg_train, Yg_train)

# モデルの予測
y_pred_g = best_rf.predict(Xg_test)

In [84]:
# F1値の出力
print("Hold-out (Xg_test) F1(macro):", f"{f1_score(Yg_test, y_pred_g, average='macro'):.4f}")

Hold-out (Xg_test) F1(macro): 0.7833


## 8. テストデータによる汎化性能の確認

最後にテストデータでモデルの汎化性能を確認しましょう。

In [85]:
# テストデータを使って予測を行いF1値を算出
y_pred_test = best_rf.predict(X_test)
print("Test (X_test) F1(macro):", f"{f1_score(Y_test, y_pred_test, average='macro'):.4f}")

Test (X_test) F1(macro): 0.7765


# 【課題外】ロジスティック回帰のパラメータチューニング
課題の想定どおりにならず、ロジスティック回帰のF1値が一番よかったため、ロジスティック回帰のパラメータチューニングを実施。

### 6.モデル選択時の結果
- LogisticRegression F1(macro): 0.7869
- DecisionTree F1(macro): 0.7783
- RandomForest F1(macro): 0.7574
- SVM F1(macro): 0.6781

In [86]:
# ロジスティック回帰のパラメータチューニング（GridSearchCV）
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import f1_score
from sklearn.linear_model import LogisticRegression

# パイプライン: 標準化 → ロジスティック回帰
pipe = Pipeline([
    ("scaler", StandardScaler()),
    ("clf", LogisticRegression(max_iter=5000, random_state=0))
])

# 探索するパラメータ
param_grid = {
    "clf__penalty": ["l2", "l1"],
    "clf__C": [0.01, 0.1, 1, 10, 100],
    "clf__solver": ["liblinear", "saga"],  # l1 と相性が良い solver を選択
    "clf__class_weight": [None, "balanced"]
}

# GridSearchCV オブジェクト作成（学習用データでCV）
gs_lr = GridSearchCV(
    estimator=pipe,
    param_grid=param_grid,
    scoring="f1_macro",
    cv=5,
    n_jobs=-1,
    refit=True,
    verbose=0
)

# 学習（X_train, Y_train は既に作成済みを想定）
gs_lr.fit(X_train, Y_train)

# CVのベスト結果
print(f"Best CV F1(macro): {gs_lr.best_score_:.4f}")
print("Best Params:", gs_lr.best_params_)

# 検証データで評価
y_valid_pred = gs_lr.predict(X_valid)
print(f"Valid F1(macro): {f1_score(Y_valid, y_valid_pred, average='macro'):.4f}")

# テストデータで最終評価
y_test_pred = gs_lr.predict(X_test)
print(f"Test F1(macro): {f1_score(Y_test, y_test_pred, average='macro'):.4f}")

Best CV F1(macro): 0.7766
Best Params: {'clf__C': 1, 'clf__class_weight': 'balanced', 'clf__penalty': 'l1', 'clf__solver': 'saga'}
Valid F1(macro): 0.7713
Test F1(macro): 0.8007
